In [42]:
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [5]:
data = pd.read_csv("donnees/bank-full.csv",sep=";")

In [6]:
data

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [10]:
for col in data.columns:

    print(data[col].value_counts())
    print('\n')

32    2085
31    1996
33    1972
34    1930
35    1894
      ... 
90       2
92       2
93       2
95       2
94       1
Name: age, Length: 77, dtype: int64


blue-collar      9732
management       9458
technician       7597
admin.           5171
services         4154
retired          2264
self-employed    1579
entrepreneur     1487
unemployed       1303
housemaid        1240
student           938
unknown           288
Name: job, dtype: int64


married     27214
single      12790
divorced     5207
Name: marital, dtype: int64


secondary    23202
tertiary     13301
primary       6851
unknown       1857
Name: education, dtype: int64


no     44396
yes      815
Name: default, dtype: int64


0        3514
1         195
2         156
4         139
3         134
         ... 
4305        1
6352        1
18881       1
14889       1
7218        1
Name: balance, Length: 7168, dtype: int64


yes    25130
no     20081
Name: housing, dtype: int64


no     37967
yes     7244
Name: loan, dtype: int6

In [14]:
data["y_num"] = data["y"].map(dict(yes=1, no=0))
data["y_num"]

0        0
1        0
2        0
3        0
4        0
        ..
45206    1
45207    1
45208    1
45209    0
45210    0
Name: y_num, Length: 45211, dtype: int64

In [24]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1,
                               test_size=0.2,
                               random_state=42)

for train_index, test_index in split.split(data, data["y_num"]):
    strat_train_set = data.loc[train_index]
    strat_test_set = data.loc[test_index]

In [25]:
from sklearn.linear_model import SGDRegressor, SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

num_attribs = ["age", "balance", "duration", "campaign", "pdays", "previous"]
cat_attribs = ["job", "marital", "education", "housing", "loan", "contact", "poutcome"]

num_pipeline = Pipeline([
        ('std_scaler', StandardScaler()),
    ])

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

X_train = full_pipeline.fit_transform(strat_train_set)
y_train = np.array(strat_train_set["y_num"])

X_test = full_pipeline.fit_transform(strat_test_set)
y_test = np.array(strat_test_set["y_num"])

model = make_pipeline(StandardScaler(),
                     SGDClassifier(max_iter=1000, loss='log'))

model.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('sgdclassifier', SGDClassifier(loss='log'))])

In [26]:
model.classes_

array([0, 1], dtype=int64)

In [27]:
print(model.predict(X_test))

print(model.predict_proba(X_test))



[0 0 0 ... 0 0 0]
[[0.98949377 0.01050623]
 [0.98448794 0.01551206]
 [0.98599767 0.01400233]
 ...
 [0.88461912 0.11538088]
 [0.94511024 0.05488976]
 [0.97884461 0.02115539]]


In [29]:
from sklearn.metrics import confusion_matrix
conf = confusion_matrix(y_test, model.predict(X_test))
conf

array([[7814,  171],
       [ 736,  322]], dtype=int64)

In [30]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, thresholds = roc_curve(y_test, model.decision_function(X_test), pos_label=2)

print(fpr)

print(tpr)

print(thresholds)

[0.00000000e+00 1.10582771e-04 1.00000000e+00]
[nan nan nan]
[10.51577007  9.51577007 -9.90367218]


E:\LOGICIELS\Anaconda3\lib\site-packages\sklearn\metrics\_ranking.py:811: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn("No positive samples in y_true, "


In [31]:
auc = auc(fpr, tpr)
auc

nan

In [32]:
print(model.score(X_test,y_test))

0.8997014265177485


In [41]:
from sklearn.metrics import classification_report, accuracy_score

print("Score :", accuracy_score(y_test,model.predict(X_test)))
print(classification_report(y_test,model.predict(X_test)))

Score : 0.8997014265177485
              precision    recall  f1-score   support

           0       0.91      0.98      0.95      7985
           1       0.65      0.30      0.42      1058

    accuracy                           0.90      9043
   macro avg       0.78      0.64      0.68      9043
weighted avg       0.88      0.90      0.88      9043

